In [1]:
import tweepy as tw
from tweepy import *
import requests
import os
import numpy as np
import pandas as pd
from azure.ai.textanalytics import TextAnalyticsClient
from azure.core.credentials import AzureKeyCredential
from azure.identity import DefaultAzureCredential


from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# your Twitter API key and API secret & Microsoft Azure API Credentials
my_api_key = os.getenv("twitter_api_key")
my_api_secret = os.getenv("twitter_secret_api_key")
language_api_key = os.getenv("azure_language")
endpoint = os.getenv("endpoint")
ta_credential = AzureKeyCredential(language_api_key)
text_analytics_client = TextAnalyticsClient(
            endpoint=endpoint, credential=ta_credential)


# authenticate
auth = tw.OAuthHandler(my_api_key, my_api_secret)
api = tw.API(auth, wait_on_rate_limit=True)

In [3]:
search_query = "#nasdaq-filter:retweets"

In [4]:
# get tweets from the API
tweets = tw.Cursor(api.search_tweets,
              q=search_query,
              lang="en",
              since="2021-09-16").items(5)

# store the API responses in a list
tweets_copy = []
for tweet in tweets:
    tweets_copy.append(tweet)
    
print("Total Tweets fetched:", len(tweets_copy))

Unexpected parameter: since


Total Tweets fetched: 5


In [5]:
# intialize the dataframe
tweets_df = pd.DataFrame()

# populate the dataframe
for tweet in tweets_copy:
    hashtags = []
    try:
        for hashtag in tweet.entities["hashtags"]:
            hashtags.append(hashtag["text"])
        text = api.get_status(id=tweet.id, tweet_mode='extended').full_text
    except:
        pass
    tweets_df = tweets_df.append(pd.DataFrame({'user_name': tweet.user.name, 
                                               'user_location': tweet.user.location,\
                                               'user_description': tweet.user.description,
                                               'user_verified': tweet.user.verified,
                                               'date': tweet.created_at,
                                               'text': text, 
                                               'hashtags': [hashtags if hashtags else None],
                                               'source': tweet.source}))
    tweets_df = tweets_df.reset_index(drop=True)

# show the dataframe
tweets_df.head(5)

,user_name,user_location,user_description,user_verified,date,text,hashtags,source
0,MR. CHEEMSINU,Cheemsville,"We are not here to take part, we are here to t...",False,2021-12-06 04:10:10+00:00,hehehe #cheems #CheemsInu making it to the big...,"[cheems, CheemsInu, shib, dogecoin, Nasdaq]",Twitter Web App
1,Crypto San@,Putrajaya,,False,2021-12-06 03:57:22+00:00,#nasdaq has gave #CheemsInu a free promotion t...,"[nasdaq, CheemsInu, doge, shiba, dogelon, Shir...",Twitter for iPhone
2,nomadhedge,,,False,2021-12-06 03:51:47+00:00,"Nasdaq was the only major U.S. index, up 0.33%...",None,Twitter Web App
3,Crypto Alien 👽,,Not financial advice. DYOR,False,2021-12-06 03:50:58+00:00,Watch out! If you make over a 100x you'll be p...,"[Nasdaq, dogecoin, SHIB, cheems]",Twitter for Android
4,Shraddha Research,,SEBi Registered Investment Adviser 💰 Social Me...,False,2021-12-06 03:47:46+00:00,"#WORLDMARKET UPDATE: \n\n#NASDAQ-15,085.47 ,-2...","[WORLDMARKET, NASDAQ, DOWJONES, HANGSENG]",Twitter Web App


In [6]:
df=tweets_df["text"]

In [7]:
df.head()

0    hehehe #cheems #CheemsInu making it to the big...
1    #nasdaq has gave #CheemsInu a free promotion t...
2    Nasdaq was the only major U.S. index, up 0.33%...
3    Watch out! If you make over a 100x you'll be p...
4    #WORLDMARKET UPDATE: \n\n#NASDAQ-15,085.47 ,-2...
Name: text, dtype: object

In [8]:
#Microsoft Azure Cognitive Text Analytics 

documents = [df[0],df[1],df[2],df[3],df[4]]

response = text_analytics_client.analyze_sentiment(documents, language="en")
result = [doc for doc in response if not doc.is_error]

for doc in result:
    print("Overall sentiment: {}".format(doc.sentiment))
    print("Scores: positive={}; neutral={}; negative={} \n".format(
        doc.confidence_scores.positive,
        doc.confidence_scores.neutral,
        doc.confidence_scores.negative,
    ))

Overall sentiment: mixed
Scores: positive=0.49; neutral=0.01; negative=0.5 

Overall sentiment: positive
Scores: positive=0.66; neutral=0.33; negative=0.01 

Overall sentiment: negative
Scores: positive=0.0; neutral=0.46; negative=0.54 

Overall sentiment: negative
Scores: positive=0.0; neutral=0.0; negative=1.0 

Overall sentiment: neutral
Scores: positive=0.0; neutral=1.0; negative=0.0 

